In [1]:
import sqlite3
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


In [2]:
db_name='dinofunworld.db'
connection = sqlite3.connect(db_name)
conn_obj = connection.cursor()

In [6]:
# 1 Write code and create a map for the three most rare crime categories in San Francisco. Give the three categories different kinds of icons.

import pandas as pd
import folium
data = pd.read_csv('Police_Department_Incidents_-_Previous_Year__2016_.csv')

rare_crime_categories = data['Category'].value_counts().tail(3).index.tolist()
rare_crimes = data[data['Category'].isin(rare_crime_categories)]


sf_map = folium.Map(location=[37.7749, -122.4194], zoom_start=12)


# Define different icons for the rare crime categories
icons = {
    rare_crime_categories[0]: 'info-sign',
    rare_crime_categories[1]: 'question-sign',
    rare_crime_categories[2]: 'exclamation-sign'
}

for _, crime in rare_crimes.iterrows():
    folium.Marker(
        location=[crime['Y'], crime['X']],
        icon=folium.Icon(icon=icons[crime['Category']])
    ).add_to(sf_map)

# Display the map
sf_map


In [13]:
#2 Write code and create a map for the events that happened on the day when the most crimes were recorded in the city. What day is it?

import pandas as pd
import folium
data = pd.read_csv('Police_Department_Incidents_-_Previous_Year__2016_.csv')

# Convert the 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'], format="%m/%d/%Y %I:%M:%S %p")

# Get the day with the most crimes recorded
most_crime_day = data['Date'].dt.date.value_counts().idxmax()

# Filter the crime data for the most crime day
most_crime_day_crimes = data[data['Date'].dt.date == most_crime_day]

# Create a map centered on San Francisco
sf_map = folium.Map(location=[37.7749, -122.4194], zoom_start=12)

# Add markers for each crime location on the most crime day
for _, crime in most_crime_day_crimes.iterrows():
    folium.Marker(
        location=[crime['Y'], crime['X']],
        popup=f"Category: {crime['Category']}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(sf_map)

# Display the map
sf_map

In [16]:
#3 Select the day and hour when the most incidents occur. Write code and create a categorized map of these incidents.

import pandas as pd
import folium
data = pd.read_csv('Police_Department_Incidents_-_Previous_Year__2016_.csv')

# Convert the 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'], format="%m/%d/%Y %I:%M:%S %p")

# Extract the day and hour from the datetime column
data['Day'] = data['Date'].dt.date
data['Hour'] = data['Date'].dt.hour

# Get the day and hour with the most incidents
most_incidents_day = data['Day'].value_counts().idxmax()
most_incidents_hour = data['Hour'].value_counts().idxmax()

# Filter the crime data for the day and hour with the most incidents
most_incidents_data = data[(data['Day'] == most_incidents_day) & (data['Hour'] == most_incidents_hour)]

# Create a map centered on San Francisco
sf_map = folium.Map(location=[37.7749, -122.4194], zoom_start=12)

# Define categories and their corresponding icons
categories = most_incidents_data['Category'].unique()

# Add markers for each incident with corresponding icons
for _, incident in most_incidents_data.iterrows():
    folium.Marker(
        location=[incident['Y'], incident['X']],
        icon=folium.Icon(icon='info-sign')
    ).add_to(sf_map)

# Display the map
sf_map

In [ ]:
# 4 Write code and create a horopleth map with generalized statistics for the city's police districts. Use the PdDistrict column for unique police district names. You can find the contours of the districts in the archive Current Police Districts.zip. This archive contains a shapefile with the boundaries of police districts. More data on zoning can be found on the open data portal of  San Francisco https://data.sfgov.org/browse?category=Geographic+Locations+and+Boundaries&page=1

import pandas as pd
import folium
import geopandas as gpd


statistics_data = pd.read_csv('Police_Department_Incidents_-_Previous_Year__2016_.csv')

shapefile_path = 'disctricts/geo_export_96bda2e6-f333-468b-af49-0fa2cc02b4e0.shp'
districts = gpd.read_file(shapefile_path)

statistics_data['PdDistrict'] = statistics_data['PdDistrict'].astype(str)

print(statistics_data['PdDistrict'])
districts = districts.merge(statistics_data, on='PdDistrict')

# Create a choropleth map centered on San Francisco
sf_map = folium.Map(location=[37.7749, -122.4194], zoom_start=12)

# Add the choropleth layer to the map
folium.Choropleth(
    geo_data=districts,
    name='choropleth',
    data=districts,
    columns=['PdDistrict', 'Statistic'],
    key_on='feature.properties.PdDistrict',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Police District Statistics'
).add_to(sf_map)

# Display the map
sf_map